## Global settings

In [ ]:
%load_ext nb_black

import os

LATEX = (
    True if os.getenv("EXPORT_LATEX") else False
)  # Whether to produce output (figures and tables) for Latex

import matplotlib.pyplot as plt
import contextily as ctx
import seaborn as sns

# sns.set_palette('deep')
cmap = sns.color_palette("deep", as_cmap=True)
cmap[1], cmap[3] = cmap[3], cmap[1]  # Swap orange and red for paper
sns.set_palette(cmap)

if LATEX:
    plt.rcParams.update(
        {
            "text.usetex": True,
            "text.latex.preamble": r"\usepackage{lmodern}",
            "font.family": "sans-serif",
            "font.size": 8,
            "figure.autolayout": True,  # tight layout
            "figure.figsize": (3.26836, 2.5),
        }
    )

## Data management

In [ ]:
import os
import dateutil
import geopandas as gp
import pandas as pd
from pyproj import CRS
import pytz

import_crs = CRS.from_epsg(4326)
tz = pytz.timezone("Europe/Berlin")


def import_gps(file: str) -> gp.GeoDataFrame:
    df = pd.read_csv(file, parse_dates=["loggingTime(txt)"])
    # Epoch time is UTC timezone
    df["Timestamp"] = pd.to_datetime(
        df["locationTimestamp_since1970(s)"], unit="s", origin="unix", utc=True
    ).dt.tz_convert(tz)
    gdf = gp.GeoDataFrame(
        df,
        geometry=gp.points_from_xy(
            df["locationLongitude(WGS84)"], df["locationLatitude(WGS84)"]
        ),
        crs=import_crs,
    )
    gdf = gdf.sort_values("Timestamp").reset_index(drop=True)
    return gdf


def import_offline_finding(file: str) -> gp.GeoDataFrame:
    df = pd.read_csv(
        file,
        parse_dates=["Date Published", "Timestamp"],
        date_parser=dateutil.parser.isoparse,
    )
    df["Timestamp"] = df["Timestamp"].dt.tz_convert(tz)
    df["Date Published"] = df["Date Published"].dt.tz_convert(tz)
    df = df.sort_values("Timestamp").reset_index(drop=True)
    gdf = gp.GeoDataFrame(
        df, geometry=gp.points_from_xy(df.Longitude, df.Latitude), crs=import_crs
    )
    return gdf


def clip_offline_finding(df_of, df_gps):
    start = df_gps["Timestamp"].min()
    end = df_gps["Timestamp"].max()
    mask = (df_of["Timestamp"] >= start) & (df_of["Timestamp"] <= end)
    return df_of[mask].reset_index(drop=True)


def project_map(gdf):
    return gdf.to_crs(epsg=3857)


def project_original(gdf):
    return gdf.to_crs(epsg=4326)


def interpolate_gps(gps_df, of_df):
    # Store old CRS
    crs = gps_df.crs
    gps_time = gps_df["Timestamp"]
    of_time = of_df["Timestamp"]
    time = gps_time.append(of_time, ignore_index=True)
    gps_interpolate = pd.DataFrame(
        {"Timestamp": time, "x": gps_df["geometry"].x, "y": gps_df["geometry"].y}
    )
    gps_interpolate = gps_interpolate.sort_values("Timestamp").reset_index(drop=True)
    gps_interpolate.index = gps_interpolate["Timestamp"]
    del gps_interpolate["Timestamp"]
    gps_interpolate = gps_interpolate.interpolate(method="time", axis=0)
    gps_interpolate = gps_interpolate.drop(gps_time)
    gps_interpolate_gs = gp.GeoDataFrame(
        gps_interpolate.index,
        geometry=gp.points_from_xy(gps_interpolate["x"], gps_interpolate["y"]),
        crs=crs,
    )
    return gps_interpolate_gs.reset_index(drop=True)

## Data import

In [ ]:
# Import GPS traces
gps_files = {
    "Walking": "data/gps/Walking-2020-07-29_11_16_05_iPhone-8.csv",
    "Train": "data/gps/Train-2020-07-29_10_40_01_iPhone-8.csv",
    "Restaurant": "data/gps/Restaurant-2020-07-29_12_11_49_iPhone-8.csv",
    "Car": "data/gps/Car_2020-08-23_iPhone-8.csv",
}
gps_traces = {name: import_gps(file) for name, file in gps_files.items()}

# Preprocess raw OF traces
of_files = []
for root, dirs, files in os.walk("data/of_raw"):
    for file in files:
        if file.endswith(".csv"):
            of_files.append(os.path.join(root, file))
if len(of_files) > 0:
    # Import all traces in single DataFrame
    of_trace = pd.concat(
        [import_offline_finding(of_file) for of_file in of_files], ignore_index=True
    )
    of_trace = of_trace.drop(columns=["Unnamed: 0"])
    # For each GPS trace, extract offline finding reports based on start and end time
    of_traces = {
        name: clip_offline_finding(of_trace, gps_trace)
        for name, gps_trace in gps_traces.items()
    }
    # Store clipped data
    for name, of_trace in of_traces.items():
        of_trace.to_csv(f"data/of/{name}.csv", index=False)

# Import OF traces
of_files = {
    "Walking": "data/of/Walking.csv",
    "Train": "data/of/Train.csv",
    "Restaurant": "data/of/Restaurant.csv",
    "Car": "data/of/Car.csv",
}
of_traces = {name: import_offline_finding(file) for name, file in of_files.items()}

# Compute interpolated GPS traces based on time index of offline finding reports
gps_interpolated_traces = {
    name: interpolate_gps(gps_trace, of_traces[name])
    for name, gps_trace in gps_traces.items()
}

## Trace statistics

In [ ]:
from geopy import distance


def gps_duration(gdf):
    return gdf["Timestamp"].max() - gdf["Timestamp"].min()


def gps_distance(gdf):
    def dist_geodesic(x):
        a = x["geometry"]
        b = x["shifted"]
        if a is None or b is None:
            return None
        # WARNING: geopy.distance.distance uses (y,x)/(lat, lon) format
        return distance.distance((a.y, a.x), (b.y, b.x)).m

    gdf2 = gdf.copy()
    gdf2["shifted"] = gdf2["geometry"].shift()
    gdf2["distance"] = gdf2.apply(dist_geodesic, axis=1)

    return gdf2["distance"].sum()


def of_reports(df):
    return len(df)


stats = {
    scenario: {
        "Distance": gps_distance(gps),
        "Duration": gps_duration(gps),
        "Num. OF reports": of_reports(of),
    }
    for scenario, gps, of in zip(
        gps_traces.keys(), gps_traces.values(), of_traces.values()
    )
}
stats_df = pd.DataFrame.from_dict(stats, orient="index")

if LATEX:
    import datetime

    def strfdelta(tdelta):
        return str(datetime.timedelta(seconds=tdelta.total_seconds()))

    stats_df["Duration"] = stats_df["Duration"].apply(strfdelta)
    print(stats_df.to_latex(float_format="{:0.0f}".format))
else:
    display(stats_df)

## Path estimation (LOWESS)

In [ ]:
import statsmodels.api as sm


def estimate_path(
    gdf: gp.GeoDataFrame, lowess_window_size=30, max_frac=1
) -> gp.GeoDataFrame:
    crs = gdf.crs
    frac = min(lowess_window_size / len(gdf), max_frac)

    time = gdf["Timestamp"]
    x = gdf.geometry.x
    y = gdf.geometry.y

    lowess_x = sm.nonparametric.lowess(x, time, return_sorted=False, frac=frac)
    lowess_y = sm.nonparametric.lowess(y, time, return_sorted=False, frac=frac)

    gdf = gp.GeoDataFrame(time, geometry=gp.points_from_xy(lowess_x, lowess_y), crs=crs)

    return gdf


of_estimated_traces = {
    name: estimate_path(of_trace) for name, of_trace in of_traces.items()
}

## Mapping

In [ ]:
def plot_traces(
    scenario,
    savefig=False,
    loc=None,
    figsize=None,
    force_figsize=True,
    with_estimated=True,
):
    of_trace = project_map(of_traces[scenario])
    of_estimated_trace = project_map(of_estimated_traces[scenario])
    gps_trace = project_map(gps_traces[scenario])

    ax = of_trace.plot(
        figsize=figsize, color="black", label="Raw OF reports", marker="."
    )
    ax.plot(gps_trace.geometry.x, gps_trace.geometry.y, linewidth=2, label="GPS trace")
    if with_estimated:
        ax.plot(
            of_estimated_trace.geometry.x,
            of_estimated_trace.geometry.y,
            linewidth=2,
            label="Estimated OF path",
        )

    # Get current figsize
    if not figsize:
        fig = plt.gcf()
        figsize = fig.get_size_inches()

    if force_figsize:
        # Crop figure to respect desired figsize
        xlen = figsize[0]
        ylen = figsize[1]
        xmin, xmax = ax.get_xlim()
        # xmax-xmin   ymax-ymin
        # --------- = ---------
        #   xlen        ylen
        yminmax = (xmax - xmin) * ylen / xlen

        ylim_min, ylim_max = ax.get_ylim()
        ylim_mid = ylim_min + (ylim_max - ylim_min) / 2
        ax.set_ylim(ylim_mid - 0.5 * yminmax, ylim_mid + 0.5 * yminmax)

    ax.legend(loc=loc)
    ax.set_yticks([])
    ax.set_xticks([])
    ax.set_frame_on(False)

    ctx.add_basemap(ax, attribution=False)

    if savefig:
        plt.savefig(
            f"plots/traces-{scenario}.pdf",
            bbox_inches="tight",
            pad_inches=0,
            metadata={"CreationDate": None},
        )


plot_traces(
    "Walking",
    savefig=LATEX,
    figsize=(3.26836, 2.3) if LATEX else None,
    loc="lower left",
)
plot_traces(
    "Restaurant",
    savefig=LATEX,
    figsize=(3.26836, 2.3) if LATEX else None,
    loc="lower right",
)
if LATEX:
    plot_traces(
        "Train",
        savefig=LATEX,
        figsize=(4, 6.69421),
        force_figsize=False,
        loc="lower center",
    )
    plot_traces(
        "Car",
        savefig=LATEX,
        figsize=(4, 11),
        force_figsize=False,
        loc="lower right",
        with_estimated=False,
    )

## Calculating distances

In [ ]:
import geopy.distance


def distance_geodesic(a_trace, b_trace):
    return pd.DataFrame(
        {
            "distance": [
                # Calculate distance in meters
                # WARNING: geopy.distance.distance uses (y,x)/(lat, lon) format
                geopy.distance.distance((a.y, a.x), (b.y, b.x)).m
                for a, b in zip(a_trace.geometry, b_trace.geometry)
            ]
        }
    )


def distance_geometric(a_trace, b_trace):
    # Warning: this produces incorrect results
    return project_map(a_trace).distance(project_map(b_trace))


def reported_accuracy(of_trace):
    df = of_trace["Accuracy"]
    df = pd.DataFrame(df.rename("distance"))
    return df


distances_raw = {
    scenario: distance_geodesic(of_trace, gps_interpolated_traces[scenario])
    for scenario, of_trace in of_traces.items()
}

distances_estimated = {
    scenario: distance_geodesic(of_estimated_trace, gps_interpolated_traces[scenario])
    for scenario, of_estimated_trace in of_estimated_traces.items()
}

distances_reported_accuracy = {
    scenario: reported_accuracy(of_trace) for scenario, of_trace in of_traces.items()
}

# Put all results in one data frame
def tidify(scenario, df, measurement):
    df["i"] = df.index
    df["measurement"] = measurement
    df["scenario"] = scenario
    return df


df_raw = [
    tidify(scenario, df, "Raw Distance") for scenario, df in distances_raw.items()
]
df_estimated = [
    tidify(scenario, df, "Estimated Path")
    for scenario, df in distances_estimated.items()
]
df_reported = [
    tidify(scenario, df, "Reported Accuracy")
    for scenario, df in distances_reported_accuracy.items()
]
df = pd.concat(df_raw + df_estimated + df_reported, axis=0, ignore_index=True)

mean = df.groupby(["scenario", "measurement"])["distance"].mean().unstack()
mean["Improvement"] = mean["Raw Distance"] / mean["Estimated Path"]

if LATEX:
    print(
        mean.to_latex(
            columns=[
                "Reported Accuracy",
                "Raw Distance",
                "Estimated Path",
                "Improvement",
            ],
            float_format="{:0.1f}".format,
        )
    )
else:
    display(mean)

## Reporting delay
We measure the delay between generating and uploading a location report

In [ ]:
def publication_delay_min(df, value: str):
    df[value] = df["Date Published"] - df["Timestamp"]
    df[value] = df[value].apply(lambda x: x.total_seconds() / 60.0)
    return df


def calculate_cdf(df, value: str):
    # CDF calculation from https://stackoverflow.com/a/54317197
    cdf = (
        df.groupby(value)[value]
        .agg("count")
        .pipe(pd.DataFrame)
        .rename(columns={value: "frequency"})
    )
    cdf["pdf"] = cdf["frequency"] / sum(cdf["frequency"])
    cdf["cdf"] = cdf["pdf"].cumsum()
    cdf = cdf.reset_index()
    return cdf


value = "Publication Delay [min]"

# Plots for individual scenarios
# _, ax = plt.subplots(1, 1, figsize=(4.1, 2.2))
# for scenario, df in of_traces.items():
#    dfa = publication_delay_min(df, value)
#    cdf = calculate_cdf(dfa, value)
#    cdf.plot(ax=ax, x=value, y='cdf', label=scenario)

df = pd.concat(of_traces.values())
df = publication_delay_min(df, value)
cdf = calculate_cdf(df, value)

ax = cdf.plot(x=value, y="cdf", label="CDF", legend=None, figsize=(3.26836, 1.4))
median = df[value].median()

ax.axvline(median, label=f"Median: {median:.1f} min", color="black")
ax.legend(loc="lower right")
ax.grid(axis="y")
ax.set_xlim(0, 3 * 60)
ax.set_xlabel("Publication delay (min)")
if LATEX:  # save plot to file
    plt.savefig(
        "plots/reporting-delay.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
        metadata={"CreationDate": None},
    )

# Identifying Top Locations

## Resamping and Clustering Location Reports

In [ ]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn import metrics

# PARAMETERS
radius = 50  # in meters
resample_interval = "20min"
min_samples = 6  # number of (resampled) samples necessary to form a cluster

# 1. run with private data (don't resample)
# X = import_offline_finding("data/of_raw/top_locations.csv")
# resample_interval = None
# 2. run with anonymized data
X = import_offline_finding("data/of/top_locations_anonymized.csv")

X.index = X["Timestamp"]  # index is generation time of report
X = X[["Latitude", "Longitude"]]
display(f"original length {len(X)}")
if resample_interval:
    X = X.resample(resample_interval, origin="start_day").mean().dropna()
    display(f"resampled length {len(X)} (resampling interval: {resample_interval})")
X = gp.GeoDataFrame(
    X, geometry=gp.points_from_xy(X["Longitude"], X["Latitude"]), crs=import_crs
)

# We need to precompute distances for DBSCAN as Eucledian distance does not work for lon/lat
def geodistance(a, b):
    return geopy.distance.distance((a[1], a[0]), (b[1], b[0])).m


x = np.array(X[["Longitude", "Latitude"]])
Xd = metrics.pairwise_distances(x, metric=geodistance)

# Compute DBSCAN
db = DBSCAN(eps=radius, min_samples=min_samples, metric="precomputed").fit(Xd)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

# Store cluster information in original data frame
X["IsCoreSample"] = core_samples_mask
X["ClusterId"] = labels
X["Date"] = X.index.to_series().dt.date
# Compute cluster statistics
C = (
    X[X["ClusterId"] != -1]
    .groupby(["ClusterId"])
    .agg(
        {
            "Longitude": "mean",
            "Latitude": "mean",
            "Date": pd.Series.nunique,
            "geometry": "count",
        }
    )
    .rename(columns={"geometry": "Samples", "Date": "Days visited"})
    .sort_values("Samples", ascending=False)
)
C["Rank"] = C.reset_index().index + 1  # TOP-X location (1-indexed)
if resample_interval:
    C["DwellTime"] = C["Samples"] * pd.Timedelta(
        resample_interval
    )  # estimate dwell time based on resampling interval
C = C.sort_index()
C = gp.GeoDataFrame(
    C, geometry=gp.points_from_xy(C["Longitude"], C["Latitude"], crs=import_crs)
)
display("Cluster statistics:")
display(C)

# Plot everything on a map
# Define colors: black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]

fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = labels == k
    # Core samples
    xy = X[(X["ClusterId"] == k) & X["IsCoreSample"]]
    if len(xy) > 0:
        project_map(xy).plot(ax=ax, color=col, marker="o", edgecolor="k")
    # Other cluster samples
    xy = X[(X["ClusterId"] == k) & ~X["IsCoreSample"]]
    if len(xy) > 0:
        project_map(xy).plot(ax=ax, color=col, marker=".", edgecolor="k")
    # Cluster centers
    xy = C[C.index == k]
    if len(xy) > 0:
        project_map(xy).plot(
            ax=ax,
            color=col,
            label=f"{int(xy.Rank)} ({int(xy.Samples)} samples)",
            marker="X",
            edgecolor="k",
            markersize=200,
        )

ctx.add_basemap(ax, attribution=False)

ax.legend(title="Cluster rank")  # Indicate cluster IDs

## Anonymize trace

**Important:** You need to run the previous cell once before so that `X` contains the clusters

In [ ]:
import random

if os.path.exists("data/of_raw/top_locations.csv"):

    Y = import_offline_finding("data/of_raw/top_locations.csv")
    Y["ClusterId"] = X["ClusterId"].reset_index(drop=True)

    # Store old coordinates
    # Y['OldLatitude'], Y['OldLongitude'] = Y['Latitude'], Y['Longitude']

    def transform(lat, lon, dlat, dlon, r_earth=6378e3):
        lat = lat + (dlat / r_earth) * (180 / np.pi)
        lon = lon + (dlon / r_earth) * (180 / np.pi) / np.cos(lat * np.pi / 180)
        return lat, lon

    # City center of London
    c_lat = 51.509865
    c_lon = -0.118092
    # Radius in meters
    r = 10000

    def random_locations(center_lat, center_lon, radius, k):
        dlat = np.array(random.choices(range(-radius, radius), k=k))
        dlon = np.array(random.choices(range(-radius, radius), k=k))
        return transform(center_lat, center_lon, dlat, dlon)

    # Randomly relocate clusters
    cids = Y["ClusterId"].unique()
    clusters = Y.groupby("ClusterId").first()
    # Generate new points
    new_lat, new_lon = random_locations(c_lat, c_lon, r, len(cids))
    # Compute Lat/Lon offsets
    dlat = new_lat - clusters["Latitude"]
    dlon = new_lon - clusters["Longitude"]
    dlon = dict(zip(cids, dlon))
    dlat = dict(zip(cids, dlat))

    Y["dlat"] = Y["ClusterId"].replace(dlat)
    Y["dlon"] = Y["ClusterId"].replace(dlon)
    Y["Latitude"] = Y["Latitude"] + Y["dlat"]
    Y["Longitude"] = Y["Longitude"] + Y["dlon"]

    # Randomize individual noise points
    noise = Y[Y["ClusterId"] == -1]
    new_lat, new_lon = random_locations(c_lat, c_lon, r, len(noise))
    Y.loc[Y["ClusterId"] == -1, "Latitude"] = new_lat
    Y.loc[Y["ClusterId"] == -1, "Longitude"] = new_lon

    Y = gp.GeoDataFrame(
        Y, geometry=gp.points_from_xy(Y["Longitude"], Y["Latitude"]), crs=import_crs
    )

    ax = project_map(Y).plot(figsize=(10, 10))
    ctx.add_basemap(ax, attribution=False)

    # Drop transformation fields and store anonymized trace
    Y = Y.drop(columns=["ClusterId", "dlat", "dlon", "geometry"])
    if not os.path.exists("data/of/top_locations_anonymized.csv"):
        Y.to_csv("data/of/top_locations_anonymized.csv", index=False)

## Calculate distance to ground truth

In [ ]:
# Input true locations here
Gdf = pd.read_csv("data/of/true_top_locations_anonymized.csv")
G = gp.GeoDataFrame(
    Gdf, geometry=gp.points_from_xy(Gdf["Longitude"], Gdf["Latitude"], crs=import_crs)
)
G.index = G["ClusterId"]
G = G.sort_values("Rank")
C = C.sort_values("Rank")

# Calculate distance cluster center
C["Accuracy"] = distance_geodesic(C, G)
C["Type"] = G["Description"]

if LATEX:
    print(
        C.to_latex(
            columns=[
                "Type",
                "Rank",
                "Accuracy",
                "Samples",
                "Days visited",
                "DwellTime",
            ],
            index=False,
        )
    )
else:
    display("Cluster statistics with distance")
    display(C)

## Plot dwell times of top locations

In [ ]:
H = X[X["ClusterId"] != -1].copy()
H["Hour of day"] = H.index.to_series().dt.hour

replace_dict = pd.Series(C["Type"], index=C.index).to_dict()
replace_rank_dict = pd.Series(C["Rank"], index=C.index).to_dict()
H["Top Location"] = H["ClusterId"].replace(replace_dict)
H["Rank"] = H["ClusterId"].replace(replace_rank_dict)
H = H.sort_values("Rank")

plt.figure(figsize=(3.26836, 1.6))
sns.histplot(
    data=H,
    x="Hour of day",
    y="Top Location",
    bins=24,
    multiple="dodge",
    hue="Top Location",
    legend=False,
)

if LATEX:
    plt.savefig(
        "plots/top-locations.pdf",
        bbox_inches="tight",
        pad_inches=0.01,
        metadata={"CreationDate": None},
    )